In [ ]:
# import numpy as np
# import scipy
# with open('./data/drug_drug_500.np', 'r') as f:
#     dd = np.load(f)

In [2]:
import pandas as pd
import scipy.sparse as sp
#import numpy as np

def load_csvfile(filename):
    return pd.read_csv(filename)

def convert_dataframe_to_sparse(matrix):
    return sp.csr_matrix(matrix)

def convert_data_to_multiarray(data):
    """
    Convert data to a multiway array by stacking matrix slices
    (also sparse version)
    """
    adj_matrices = []
    adj_sp_matrices = []
    unique_seffects = data.iloc[:, 2].unique()
    # build a huge zeros matrix for all nodes
    columns = (data.iloc[:, i] for i in range(2))
    df = pd.crosstab(*columns)
    #df = pd.crosstab(data.a, data.b)
    idx = df.columns.union(df.index)
    adj_matrix_all = df.reindex(index=idx, columns=idx, fill_value=0)

    # for each disease get all interaction rows
    # and construct a DxDx#unique_seffects
    #print len(unique_seffects.tolist())

    for se in unique_seffects.tolist():
        adj_matrix_all[adj_matrix_all>0] = 0
        subdata = data.loc[data.iloc[:, 2] == se]
        #columns = (subdata.iloc[:, i] for i in range(3))
        for c in subdata.values:
            row = c[0]
            column = c[1]
            adj_matrix_all.at[row, column] = 1
        adj_matrices.append(adj_matrix_all.values)
        sparse_matrix = convert_dataframe_to_sparse(adj_matrix_all)
        adj_sp_matrices.append(sparse_matrix)
    #multiarray = np.dstack(adj_matrices)

    return adj_sp_matrices

datafile = load_csvfile("./data/bio-decagon-combo.csv")
multiarray = convert_data_to_multiarray(datafile)

In [ ]:
from sklearn.preprocessing import normalize
import tqdm
random_state = 42
np.random.seed(random_state)

def bpr(X, rank=10, alpha=0.05, lambdaA=0.0001, lambdaR=0.00001, max_iter=500, init={'key':'normal', 'mean':0,'std':0.01}):
    """
    X: list of matices, mandatory
    rank:
    lambdaA : float, optional
        Regularization parameter for A factor matrix. 0 by default
    lmbdaR : float, optional
        Regularization parameter for R_k factor matrices. 0 by default
    lmbdaV : float, optional
    init : string, optional
        'normal' initializes the factor matrices randomly.
    compute_fit : boolean, optional
        If true, compute the fit of the factorization compared to X.
        For large sparse tensors this should be turned of. None by default.
    """
    k = len(X)
    dim, _ = X[0].shape
    A, R = initialize_matrices(X, rank, dim, k, init)
    for iter_ in tqdm.tqdm(xrange(max_iter)):
        #P = reconstruct(A, R)
        i, j, r, r_neg = draw_samples(X)
        #print P.sum()
        a_j_R_pos = np.dot(R[:,:,r], A[j,:].T)
        a_j_R_neg = np.dot(R[:,:,r_neg], A[j,:].T)
        y_hat = np.dot(A[i,:], a_j_R_pos) - np.dot(A[i,:], a_j_R_neg)
        #y_hat = predict(A,R,i,j,r) - predict(A,R,i,j,r_neg)
        delta = 1-sigmoid(y_hat)
        #print delta
        delta_alpha_i = np.dot(A[i,:], R[:,:,r])  - np.dot(A[i,:], R[:,:,r_neg])
        delta_alpha_j = a_j_R_pos - a_j_R_neg
        delta_rel = np.outer(A[i,:], A[j,:])
        A[i,:] = A[i,:] + alpha*(delta*delta_alpha_i - lambdaA*A[i,:])
        A[j,:] = A[j,:] + alpha*(delta*delta_alpha_j - lambdaA*A[j,:])
        R[:,:,r] = R[:,:,r] + alpha*(delta*delta_rel - lambdaR*R[:,:,r])
        R[:,:,r_neg] = R[:,:,r_neg] + alpha*(-delta*delta_rel - lambdaR*R[:,:,r_neg])
    return A, R
        
def reconstruct(A, R):
    P = np.zeros((A.shape[0], A.shape[0], R.shape[2]))
    for k in range(len(R)):
        P[:, :, k] = np.dot(A, np.dot(R[:,:,k], A.T))
    return P
    
def predict(A, R, i, j, r):
    #print A[i,:].shape, R[:,:,r].shape, A[j,:].T.shape
    return  np.dot(A[i,:], np.dot(R[:,:,r], A[j,:].T))
	
    
def draw_negative_sample(r, N):
    r_dot = np.random.randint(N) # sample random index
    while r_dot==r:
        r_dot = np.random.randint(N) # repeat while the same index is sampled
    return r_dot

def sigmoid(x):
    if x > 20:
        return 1
    elif x < 0:
        return 0
    else:
        sigm = 1.0 / (1.0 + np.exp(-x))   
        if sigm > 0.5:
            #print 'ole'
            return sigm
        else:
            return 0
    
def initialize_matrices(X, rank, dim, k, init):
    """
    Initialize the A, R matrices.
    """
    if init['key'] == 'normal':
        A = np.random.uniform(init['mean'], init['std'], size=(dim, rank))
        R = np.random.uniform(init['mean'], init['std'], size=(rank, rank, k))
    elif init['key'] == 'nvecs':
        S = sparse.csr_matrix((dim, dim))
        for i in range(k):
            S = S + X[i]
            S = S + X[i].T
        _, A = sparse.linalg.eigsh(sparse.csr_matrix(S, shape=(dim, dim)), rank)
        A = array(A, dtype=dtype)
    else:
        print('Initialization method not found!')
        print(init)
        raise KeyError
    return A, R

def draw_samples(X, N=1):
    """
    Input:
        X: list of matices, mandatory
    Output:
        pos: tuple, int
        indices of i,j,r positive triples
        neg: tuple, int
        indices of i,j,r neg triples
    """
    if N == 1:
        r = np.random.randint(len(X))
        nnz = np.transpose(np.nonzero(X[r]))
        i, j = nnz[np.random.choice(nnz.shape[0])]
        neg = np.random.choice([k for k in xrange(len(X)) if X[k][i,j]==0])
    else:
        i = j = r = neg = []
        for xx in xrange(N):
            r.append(np.random.randint(len(X)))
            nnz = np.transpose(np.nonzero(X[r]))
            ii, jj = nnz[np.random.choice(nnz.shape[0])]
            i.append(ii)
            j.append(jj)
            neg.append(np.random.choice([k for k in xrange(len(X)) if X[k][i,j]==0]))
    return i, j, r, neg
A, R = bpr(multiarray, 10, max_iter=100)

In [ ]:
def to_tensor(multiarray_sparse, sample=None):
    marrays = []
    if sample:
        sorted_m = sorted(multiarray_sparse, key=sp.csr_matrix.getnnz)
        for m in sorted_m[-sample:]:
            marrays.append(m.toarray())
        T = np.array(marrays).T
    else:
        for m in multiarray_sparse:
            marrays.append(m.toarray())
        T = np.array(marrays).T
    return T

def normalize_predictions(P, e, k):
    print(P.shape)
    for a in range(e):
        for b in range(e):
            nrm = np.linalg.norm(P[a, b, :k])
            if nrm != 0:
                # round values for faster computation of AUC-PR
                P[a, b, :k] = np.round_(P[a, b, :k] / nrm, decimals=3)
    return P

data = to_tensor(multiarray, sample=500)
e, k = data.shape[0], data.shape[2]
print("Dataset size: ", data.shape)
SZ = e * e * k
# T for rescal
T = [sp.lil_matrix(data[:, :, i]) for i in range(k)]

A, R = bpr(T, 10, max_iter=10000)
n = A.shape[0]
P = np.zeros((n, n, R.shape[2]))
for k_i in range(R.shape[2]):
    P[:, :, k_i] = np.dot(A, np.dot(R[:,:,k_i], A.T))
P = normalize_predictions(P, e, k)
print("Error: ", np.linalg.norm(data-P))

In [ ]:

n = A.shape[0]
P = np.zeros((n, n, R.shape[2]))
for k_i in range(R.shape[2]):
    P[:, :, k_i] = np.dot(A, np.dot(R[:,:,k_i], A.T))
P = normalize_predictions(P, e, k)
print("Error: ", np.linalg.norm(data-P))
print P[:,:,0].sum()

In [1]:
import autograd.numpy as np
from autograd import grad

def sigmoid(x):
    return 0.5 * (np.tanh(x / 2.) + 1)

def logistic_predictions(weights, inputs):
    # Outputs probability of a label being true according to logistic model.
    return sigmoid(np.dot(inputs, weights))

def training_loss(weights):
    # Training loss is the negative log-likelihood of the training labels.
    preds = logistic_predictions(weights, inputs)
    label_probabilities = preds * targets + (1 - preds) * (1 - targets)
    return -np.sum(np.log(label_probabilities))

# Build a toy dataset.
inputs = np.array([[0.52, 1.12,  0.77],
                   [0.88, -1.08, 0.15],
                   [0.52, 0.06, -1.30],
                   [0.74, -2.49, 1.39]])
targets = np.array([True, True, False, True])

# Define a function that returns gradients of training loss using Autograd.
training_gradient_fun = grad(training_loss)

# Optimize weights using gradient descent.
weights = np.array([0.0, 0.0, 0.0])
print("Initial loss:", training_loss(weights))
for i in range(100):
    weights -= training_gradient_fun(weights) * 0.01

print("Trained loss:", training_loss(weights))

('Initial loss:', 2.772588722239781)
('Trained loss:', 1.067270675787016)


In [33]:
def draw_samples(X, N=1):
    """
    Input:
        X: list of matices, mandatory
    Output:
        pos: tuple, int
        indices of i,j,r positive triples
        neg: tuple, int
        indices of i,j,r neg triples
    """
    if N == 1:
        r = np.random.randint(len(X))
        nnz = np.transpose(np.nonzero(X[r]))
        i, j = nnz[np.random.choice(nnz.shape[0])]
        neg = np.random.choice([k for k in xrange(len(X)) if X[k][i,j]==0])
    else:
        i = j = r = neg = []
        for xx in xrange(N):
            r.append(np.random.randint(len(X)))
            nnz = np.transpose(np.nonzero(X[r[-1]]))
            ii, jj = nnz[np.random.choice(nnz.shape[0])]
            i.append(ii)
            j.append(jj)
            neg.append(np.random.choice([k for k in xrange(len(X)) if X[k][i[-1],j[-1]]==0]))
    return i, j, r, neg
N = 100
i, j, r, r_neg = draw_samples(multiarray, N)

In [24]:
nnz

NameError: name 'nnz' is not defined

IndexError: index out of bounds

In [44]:
def to_tensor(multiarray_sparse, sample=None):
    marrays = []
    if sample:
        sorted_m = sorted(multiarray_sparse, key=sp.csr_matrix.getnnz)
        for m in sorted_m[-sample:]:
            marrays.append(m.toarray())
        T = np.array(marrays).T
    else:
        for m in multiarray_sparse:
            marrays.append(m.toarray())
        T = np.array(marrays).T
    return T


data = to_tensor(multiarray, sample=500)
e, k = data.shape[0], data.shape[2]
print("Dataset size: ", data.shape)
SZ = e * e * k
# T for rescal
del T

('Dataset size: ', (645, 645, 500))


In [43]:
T[:,:,0]

TypeError: list indices must be integers, not tuple

In [38]:
def predict(A, R, i, j, r):
    #print A[i,:].shape, R[:,:,r].shape, A[j,:].T.shape
    return  np.dot(A[i,:], np.dot(R[:,:,r], A[j,:].T))

def BPR_LOSS(A,R,X, batch_size=100, lambdaA=0.0001, lambdaR=0.00001):
    pairs = []
    y_hat = 0
    for b in xrange(batch_size):
        i, j, r, r_neg = draw_samples(X)
        #a_j_R_pos = np.dot(R[:,:,r], A[j,:].T)
        #a_j_R_neg = np.dot(R[:,:,r_neg], A[j,:].T)
        #y_hat += sigmoid(np.dot(A[i,:], a_j_R_pos) - np.dot(A[i,:], a_j_R_neg))
        y_hat += predict(A,R,i,j,r) - predict(A,R,i,j,r_neg)
    return -y_hat #- lambdaA*np.linalg.norm(A) -  lambdaA*np.linalg.norm(R))
#BPR_LOSS(A,R,multiarray)

rank=10
init={'key':'normal', 'mean':0,'std':0.01}

grad_a = grad(BPR_LOSS, 0)
grad_r = grad(BPR_LOSS, 1)

k = len(multiarray)
dim, _ = multiarray[0].shape
A, R = initialize_matrices(rank, dim, k, init)
print("Initial loss:", BPR_LOSS(A,R,multiarray))
for i in range(100):
    A += grad_a(A,R,multiarray) * 0.01
    R += grad_r(A,R,multiarray) * 0.01
    print("Loss at %d : %0.3f" % (i,BPR_LOSS(A,R,multiarray)))

print("Trained loss:", training_loss(weights))

('Initial loss:', -1.5487267262981368e-05)
Loss at 0 : -0.000


KeyboardInterrupt: 

In [ ]:
 k = len(X)
    dim, _ = X[0].shape
    A, R = initialize_matrices(rank, dim, k, init)
    for iter_ in tqdm.tqdm(xrange(max_iter)):
        #P = reconstruct(A, R)
        i, j, r, r_neg = draw_samples(X)
        #print P.sum()
        a_j_R_pos = np.dot(R[:,:,r], A[j,:].T)
        a_j_R_neg = np.dot(R[:,:,r_neg], A[j,:].T)
        y_hat = np.dot(A[i,:], a_j_R_pos) - np.dot(A[i,:], a_j_R_neg)
        #y_hat = predict(A,R,i,j,r) - predict(A,R,i,j,r_neg)
        delta = 1-sigmoid(y_hat)
        #print delta
        delta_alpha_i = np.dot(A[i,:], R[:,:,r])  - np.dot(A[i,:], R[:,:,r_neg])
        delta_alpha_j = a_j_R_pos - a_j_R_neg
        delta_rel = np.outer(A[i,:], A[j,:])
        A[i,:] = A[i,:] + alpha*(delta*delta_alpha_i - lambdaA*A[i,:])
        A[j,:] = A[j,:] + alpha*(delta*delta_alpha_j - lambdaA*A[j,:])
        R[:,:,r] = R[:,:,r] + alpha*(delta*delta_rel - lambdaR*R[:,:,r])
        R[:,:,r_neg] = R[:,:,r_neg] + alpha*(-delta*delta_rel - lambdaR*R[:,:,r_neg])
    return A, R